# Join Statements

## Introduction

In this lab, you'll practice your knowledge on Join statements.

## Objectives

You will be able to:
- Write queries that make use of various types of Joins
- Join tables using foreign keys

## CRM Schema

In almost all cases, rather then just working with a single table we will typically need data from multiple tables. 
Doing this requires the use of **joins ** using shared columns from the two tables. 

In this lab, we'll use the same Customer Relationship Management (CRM) database we used in our lecture before!
<img src='Database-Schema.png' width=550>

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [2]:
#Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

## Display the names of all the employees in Boston.

In [3]:
!ls

data.sqlite
Database-Schema.png
index.ipynb
README.md
venn.png


In [88]:
cur.execute("""
SELECT lastName, firstName 
FROM employees e
    join offices o
    using(officeCode)
WHERE o.city = "Boston"
;""")
df = pd.DataFrame(cur.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in cur.description]
df.head()

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Do any offices have no employees?

In [89]:
#Your code here
# All Offices
cur.execute("""
SELECT DISTINCT officeCode 
FROM employees e
    join offices o
    using(officeCode)
GROUP BY(officeCode)
;""")
df_oC = pd.DataFrame(cur.fetchall()) #Take results and create dataframe
df_oC.columns = [i[0] for i in cur.description]

#Offices with employees
cur.execute("""
SELECT officeCode
FROM offices o
GROUP BY(officeCode)
;""")
df_oe = pd.DataFrame(cur.fetchall()) #Take results and create dataframe
df_oe.columns = [i[0] for i in cur.description]

print("It is", df_oe.equals(df_oC), "that all offices have employees.")

It is True that all offices have employees.


## Write 3 Questions of your own and answer them

In [85]:
# Answers will vary
# Which product is most popular? How many Sales?
cur.execute("""
SELECT productName, SUM(quantityOrdered) AS Sales
FROM products p
    JOIN orderdetails od
        USING(productcode)
GROUP BY productName
ORDER BY Sales DESC
LIMIT 1
;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productName,Sales
0,1992 Ferrari 360 Spider red,1808


In [87]:
# Your code here
# Which product generates the least revenue?
cur.execute("""
SELECT productName, SUM(quantityOrdered*priceEach) AS Revenue
FROM products p
    JOIN orderdetails od
        USING(productcode)
GROUP BY productName
ORDER BY Revenue
LIMIT 1
;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productName,Revenue
0,1939 Chevrolet Deluxe Coupe,28052.94


In [96]:
# Your code here
# Which office city has the most sales?
cur.execute("""
SELECT os.city, sum(quantityOrdered) AS Sales
FROM customers c
    JOIN orders o
        USING(customerNumber)
    JOIN orderdetails od
        USING(orderNumber)
    JOIN products p
        USING(productCode)
    JOIN employees e
        ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN offices os
        USING(officeCode)
GROUP BY os.city
ORDER BY(Sales) DESC
;""")
df = pd.DataFrame(cur.fetchall())
df.columns= [i[0] for i in cur.description]
df.head()

,city,Sales
0,Paris,33887
1,San Francisco,15910
2,London,15691
3,Sydney,12878
4,NYC,12439


## Level Up: Display the names of each product each employee has sold.

In [35]:
# Your code here
# Multiple joins (customers-orders-orderdetails-products-employees)
cur.execute("""
SELECT lastName, firstName, productName
FROM customers c
    JOIN orders o
        USING(customerNumber)
    JOIN orderdetails od
        USING(orderNumber)
    JOIN products p
        USING(productCode)
    JOIN employees e
        ON e.employeeNumber = c.salesRepEmployeeNumber
ORDER BY(salesRepEmployeeNumber)
;""")
df = pd.DataFrame(cur.fetchall())
df.columns= [i[0] for i in cur.description]
df.head()

,lastName,firstName,productName
0,Jennings,Leslie,1958 Setra Bus
1,Jennings,Leslie,1940 Ford Pickup Truck
2,Jennings,Leslie,1939 Cadillac Limousine
3,Jennings,Leslie,1996 Peterbilt 379 Stake Bed with Outrigger
4,Jennings,Leslie,1968 Ford Mustang


## Level Up: Display the Number of Products each Employee Has sold

In [69]:
#Your code here
cur.execute("""
SELECT employeeNumber, SUM(quantityOrdered) AS Employee_Sales
FROM customers c
    JOIN orders o
        USING(customerNumber)
    JOIN orderdetails od
        USING(orderNumber)
    JOIN products p
        USING(productCode)
    JOIN employees e
        ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY employeeNumber
ORDER BY Employee_Sales DESC
;""")
df = pd.DataFrame(cur.fetchall())
df.columns= [i[0] for i in cur.description]
df
#pd.DataFrame(df.pivot)
#index='lastName')#, values = 'quantityOrdered')

,employeeNumber,Employee_Sales
0,1370,14231
1,1165,11854
2,1401,9290
3,1501,8205
4,1504,7486
5,1323,7423
6,1612,6632
7,1611,6246
8,1337,6186
9,1216,5561


## Summary

Congrats! You now know how to use Join statements, along with leveraging your foreign keys knowledge!